# 1. Install Tools

In [1]:
!python -m pip install mahotas

# 2. Preprocessing

In [3]:
import os
import numpy as np
import pylab
import mahotas as mh

from ipywidgets import interact, interactive
import ipywidgets as widgets

In [4]:
def getImage(directory: str, name: str) -> np.uint8: 
    image = None 
    imageFind = None
    try:
        data_directory = os.path.join('.', directory)
        imageFind = os.path.join(data_directory, name)
        # Find File
        image = mh.imread(imageFind)
    except:
        return False, image, imageFind;
    
    
    return True, image, imageFind;

In [5]:
def convertType(image, newType='uint8') -> np.uint8:
    return image.astype(newType)

In [6]:
def getSpecs(image: np.uint8):
    # Show Files
    print (image.shape)
    print (image.dtype)
    print (image.max())
    print (image.min())
    return;

In [7]:
def scaleMask(image: np.uint8) -> np.uint8:
    image[960:1024, 0:200] = 0
    return image

# 3. Image Processing

In [8]:
def threshold(image) -> int:
    return mh.thresholding.otsu(image)

In [9]:
def gaussian(image, kernel) -> float:
    return mh.gaussian_filter(image, 3)

In [13]:
def processing(Directory, ImageName, ThresholdSlider, GaussianSlider, sizeSlider):
    found, image, imageFind = getImage(Directory, ImageName)
    
    if not found:
        print("File Not Found")
        return;
    else:
        print(imageFind + " Found!")

    # preprocessing
    image = convertType(image)
    scaleMask(image)
    
    # gaussian
    smoothedImage = convertType(gaussian(image, GaussianSlider))
    
    #thresholding
    labeled, nr_objects = mh.label(smoothedImage > ThresholdSlider)
    
    # size
    sizes = mh.labeled.labeled_size(labeled)
    too_big = np.where(sizes < sizeSlider)
    too_big = np.asarray(too_big)    
    labeled = mh.labeled.remove_regions(labeled, too_big) # remove
    
    
    

    labeled, nr_objects = mh.label(convertType(labeled))
    
    print ("Cell Count: " + str(nr_objects))
    
    
    rmax = mh.regmax(labeled)
    pylab.imshow(rmax*255)
    
    pylab.jet()
    pylab.show()
    
    return;

In [14]:
ThresholdSlider = widgets.IntSlider(
    value=10,
    min=10,
    max=255,
    step=1,
    description='Thresholding',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
GaussianSlider = widgets.IntSlider(
    value=1,
    min=1,
    max=16,
    step=1,
    description='Kernel Gauss',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
sizeSlider = widgets.IntSlider(
    value=1,
    min=10,
    max=1500,
    step=1,
    description='Size',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

Directory = widgets.Text(
    description='Directory',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
ImageName = widgets.Text(
    description='Image Name',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


# 4. Formation

In [15]:
interact(processing, Directory=Directory, ImageName=ImageName, ThresholdSlider=ThresholdSlider, GaussianSlider=GaussianSlider, sizeSlider=sizeSlider)

interactive(children=(Text(value='', description='Directory'), Text(value='', description='Image Name'), IntSl…

<function __main__.processing(Directory, ImageName, ThresholdSlider, GaussianSlider, sizeSlider)>